In [1]:
import timeit
import numpy as np
from time import time

In [2]:
from numba import cuda
import numba as nb
import math

In [ ]:
print(cuda.gpus)

In [ ]:
cuda.select_device(0)

In [ ]:
# %%time
# Initialize the data arrays
A = np.asfortranarray(np.random.randn(32*2, 32*3))
B = np.asfortranarray(np.random.randn(3*32, 32))
C = np.asfortranarray(np.random.randn(32*2, 32))

In [ ]:
%%time
# Copy the arrays to the device
A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)
C_global_mem = cuda.to_device(C)

# Allocate memory on the device for the result
# C_global_mem = cuda.device_array((32*2, 32))

In [ ]:
%%time
# Configure the blocks
threadsperblock = (16,16)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

In [ ]:
blockspergrid

In [ ]:
(A.nbytes+B.nbytes+np.dot(A,B).nbytes)/1024/1024

In [ ]:
# CUDA kernel
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

In [ ]:
%%time
matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

In [ ]:
%%time
C = C_global_mem.copy_to_host()
print(C.shape)

In [ ]:
%%time
np.allclose(C,np.dot(A,B))

In [ ]:
np.dot(A,B)

In [ ]:
C

In [ ]:
%time
@cuda.jit
def my_kernel_2D(io_array):
    x, y = cuda.grid(2)
    if x<io_array.shape[0] and y<io_array.shape[1]:
        io_array[x,y]*=8

data = np.ones((16, 16))
data_glob=cuda.to_device(data)
threadsperblock = (32, 32)
blockspergrid_x = math.ceil(data.shape[0] / threadsperblock[0])
blockspergrid_y = math.ceil(data.shape[1] / threadsperblock[1])
blockspergrid = (blockspergrid_x, blockspergrid_y)
my_kernel_2D[blockspergrid, threadsperblock](data_glob)
print(data_glob.copy_to_host())

In [ ]:
A.shape,B.shape

In [ ]:
TPB=16
@cuda.jit
def fast_matmul(A,B,C):
    sA=cuda.shared.array(shape=(TPB,TPB),dtype=nb.float32)
    sB=cuda.shared.array(shape=(TPB,TPB),dtype=nb.float32)
    x,y=cuda.grid(2)
    tx=cuda.threadIdx.x
    ty=cuda.threadIdx.y
    if x>=C.shape[0] and y>=C.shape[1]:
        return
    tmp=0
    for i in range(A.shape[1]//TPB):  # for number of blocks
        sA[tx,ty]=A[x,ty+i*TPB]       # preLoad data into shared memory
        sB[tx,ty]=B[tx+i*TPB,y]
        
        cuda.syncthreads()            # wait for loading
        for j in range(TPB):
            tmp+=sA[tx,j]*sB[j,ty]
            cuda.syncthreads()            # wait for computation
    C[x,y]=tmp

In [ ]:
%%time
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)

In [ ]:
%%time
C = C_global_mem.copy_to_host()
print(C.shape)

In [ ]:
C

In [6]:
import pycuda

In [4]:
cudlib=ctypes.CDLL('libcublas.so')

In [5]:
N=128

In [ ]:
A = np.array(np.arange(N ** 2, dtype=np.float32).reshape(N, N), order='F')
B = np.array(np.arange(N) + 10, dtype=A.dtype, order='F')
D = np.zeros_like(A, order='F')

In [ ]:
np.asfortranarray(A).ctypes.data

In [ ]:
D

In [ ]:
cudlib.cublasSdot

In [ ]:
A_global_mem.device_ctypes_pointer

In [ ]:
cudlib.cublasSgemm('N', 'N', N, N, N, 1, A_global_mem.device_ctypes_pointer, B_global_mem.device_ctypes_pointer, 1, C_global_mem.device_ctypes_pointer)

In [1]:
import ctypes
import numpy as np

In [2]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return weights, bias

In [3]:
w0,b0=init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=128)

In [4]:
inp=np.random.randn(128,64,64,32)

In [5]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=0
stride=[1,1]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding)//stride[0]+1),((ipp.shape[3]-ksz+2*padding)//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding
padded=np.zeros((batches,d,row,col))
padded[:,:,padding:-padding,padding:-padding]=ipp

In [6]:
img=padded[0]

In [7]:
# %%timeit
window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
# bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker)
# output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [11]:
%%time
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)
output+=b0
ans2=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 4.61 s, sys: 129 ms, total: 4.74 s
Wall time: 1.26 s


In [9]:
ans2.nbytes/1024/1024

512.0

In [10]:
%%time
output=np.empty((batches,*ind.shape))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=img.take(ind)
# output+=b0
# ans2=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 361 ms, sys: 46.8 ms, total: 407 ms
Wall time: 407 ms


In [12]:
ind.shape,kern.shape,output.shape,padded.shape

((4096, 288), (288, 128), (128, 4096, 128), (128, 32, 66, 66))

In [18]:
a=output[::20].reshape(-1,128)
b=padded[::20].reshape(128,-1)
(a.nbytes+b.nbytes)/1024/1024

35.4443359375

In [33]:
%%time
c=np.dot(a,b)

CPU times: user 4.34 s, sys: 276 ms, total: 4.61 s
Wall time: 1.39 s


In [19]:
c=np.empty((a.shape[0],b.shape[1]))
c.nbytes/1024/1024

1667.53125

In [11]:
a=(np.arange(30)+11).reshape(6,5,order='F').astype(np.float32)  #mxk

In [12]:
b=(np.arange(20)+11).reshape(5,4,order='F').astype(np.float32)  #kxn

In [13]:
c=(np.arange(24)*0).reshape(6,4,order='F').astype(np.float32)  #mxn

In [171]:
a=np.asfortranarray(img.take(ind).astype(np.float32))
b=np.asfortranarray(kern.astype(np.float32))
c=np.asfortranarray(np.empty((4096,128)).astype(np.float32))

In [50]:
c

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [70]:
np.asfortranarray(a).ctypes.data

94390980909520

In [20]:
al,bet=1,0

In [21]:
# al*np.dot(a,b)+bet*c

In [22]:
sgemm=ctypes.CDLL('libsgemm.so')

In [35]:
%%time
sgemm.gemm(ctypes.c_void_p(a.ctypes.data),ctypes.c_void_p(b.ctypes.data),ctypes.c_void_p(c.ctypes.data),a.shape[0],b.shape[0],c.shape[1],ctypes.c_float(al),ctypes.c_float(bet))

CPU times: user 888 ms, sys: 358 ms, total: 1.25 s
Wall time: 1.24 s


0

In [159]:
c

array([[-0.00998211, -0.15539624, -0.0916025 , ..., -0.06454306,
        -0.11045393, -0.03924982],
       [ 0.02346075,  0.07263432, -0.03241877, ..., -0.13095035,
        -0.21292068, -0.02834077],
       [ 0.2574331 ,  0.02906782,  0.0503069 , ..., -0.09986894,
         0.11187343, -0.11246274],
       ...,
       [-0.00362097,  0.03146717, -0.13846727, ..., -0.03131997,
        -0.08544292,  0.04434009],
       [ 0.28387368,  0.12797177,  0.11094224, ...,  0.12037938,
        -0.11533047, -0.27713284],
       [-0.06219382,  0.00298618, -0.04466361, ...,  0.05428534,
         0.18927112, -0.17992361]], dtype=float32)

In [160]:
np.dot(a,b)

array([[-0.00998211, -0.15539624, -0.0916025 , ..., -0.06454306,
        -0.11045393, -0.03924982],
       [ 0.02346075,  0.07263432, -0.03241877, ..., -0.13095035,
        -0.21292068, -0.02834077],
       [ 0.2574331 ,  0.02906782,  0.0503069 , ..., -0.09986894,
         0.11187343, -0.11246274],
       ...,
       [-0.00362097,  0.03146717, -0.13846727, ..., -0.03131997,
        -0.08544292,  0.04434009],
       [ 0.28387368,  0.12797177,  0.11094224, ...,  0.12037938,
        -0.11533047, -0.27713284],
       [-0.06219382,  0.00298618, -0.04466361, ...,  0.05428534,
         0.18927112, -0.17992361]], dtype=float32)